In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import os
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import MaxNLocator


In [ ]:
# data_path = "../results/all_performance_summary.csv"
# data_path = "../results-O2-rvv/all_performance_summary.csv"
rv_riscv64gc_data_path = "../results-O2-rv-riscv64gc/all_performance_summary.csv"
rv_riscv64gcv_data_path = "../results-O2-rvv-riscv64gcv-old/all_performance_summary.csv"
# RVV before 2025.04.04
rvv_riscv64gcv_old_data_path = "../results-O2-rvv-riscv64gcv-old/all_performance_summary.csv"
# RVV after 2025.04.04
rvv_riscv64gcv_data_path = "../results-O2-rvv-riscv64gcv-0404/all_performance_summary.csv"


# Read the CSV data
# df = pd.read_csv(data_path)
# df_rv = pd.read_csv(rv_data_path)
df_rv_riscv64gc = pd.read_csv(rv_riscv64gc_data_path)
df_rv_riscv64gcv = pd.read_csv(rv_riscv64gcv_data_path)
df_rvv_riscv64gcv_old = pd.read_csv(rvv_riscv64gcv_old_data_path)
df_rvv_riscv64gcv = pd.read_csv(rvv_riscv64gcv_data_path)


In [ ]:
# L1D 64KiB

riscv64gcv_l1d64_data_path = "../results-O2-riscv64gcv-l1d64/all_performance_summary.csv"
df_riscv64gcv_l1d64 = pd.read_csv(riscv64gcv_l1d64_data_path)


In [ ]:
# L1D and L2 Latency / 2

riscv64gcv_cachelatdiv2_data_path = "../results-O2-rvv-riscv64gcv-cachelatdiv2/all_performance_summary.csv"
df_riscv64gcv_cachelatdiv2 = pd.read_csv(riscv64gcv_cachelatdiv2_data_path)


In [ ]:
from gem5_metrics import metrics

print("metrics: ", metrics.values())

df_labels = df_rvv_riscv64gcv.columns
print("df_labels: ", df_labels)

vlen_list = df_rvv_riscv64gcv['vlen'].unique()
vlen_list.sort()
print("vlen_list: ", vlen_list)

top_k_list = df_rvv_riscv64gcv['top_k'].unique()
top_k_list.sort()
print("top_k_list: ", top_k_list)

dimension_list = df_rvv_riscv64gcv['dimension'].unique()
dimension_list.sort()
print("dimension_list: ", dimension_list)


In [ ]:
def set_fonts():
    # 请根据您的系统调整字体路径
    try:
        font_ch = FontProperties(fname="/usr/share/fonts/SIMSUN.TTC", size=12) # 尝试加载宋体
    except FileNotFoundError:
        print("Warning: SIMSUN.TTC not found. Using default font for Chinese.")
        font_ch = FontProperties(size=12) # 使用默认字体
    font_en = FontProperties(family="Times New Roman", size=12)
    return font_ch, font_en


In [ ]:
markers = [
    "o",
    "D",
    "s",
    "^",
    "v",
    "<",
    ">",
    "p",
    "*",
    "h",
    "+",
    "x",
    "|",
    "_",
]

linestyles = [
    "--",
    "-",
    "-.",
    ":",
]


In [ ]:
# 定义输出目录
output_dir = 'plots_output'
os.makedirs(output_dir, exist_ok=True)


In [ ]:
zh_font, en_font = set_fonts()

plot_configs = [
    {"df": df_rvv_riscv64gcv, "workload": "rvv", "label": "RVV"},
    {"df": df_rv_riscv64gcv, "workload": "rv", "label": "RV"},
    # {'df': df_rvv_riscv64gcv, 'workload': 'rvv', 'label': 'rvv-riscv64gcv'},
    # {'df': df_rv_riscv64gcv, 'workload': 'rv', 'label': 'rv-riscv64gcv'},
    # {'df': df_rvv_riscv64gcv_old, 'workload': 'rvv', 'label': 'rvv-riscv64gcv-old'},
    # {'df': df_rv_riscv64gc, 'workload': 'rv', 'label': 'rv-riscv64gc'},
    # {"df": df_riscv64gcv_l1d64, "workload": "rvv", "label": "RVV-L1D64KB"},
    # {"df": df_riscv64gcv_l1d64, "workload": "rv", "label": "RV-L1D64KB"},
    # {"df": df_riscv64gcv_cachelatdiv2, "workload": "rvv", "label": "RVV-Latency/2"},
    # {"df": df_riscv64gcv_cachelatdiv2, "workload": "rv", "label": "RV-Latency/2"},
]


In [ ]:
# Plotting with vlen as x-axis
# for metric_label in ['SimSecond', 'CPU_IPC', 'L1d_TotMissRate', 'L1d_TotAvgMissLat', 'L2_TotMissRate', 'L2_TotAvgMissLat']:
# for metric_label in ['SimSecond']:
# for metric_label in ["L1d_TotMissRate", "L2_TotMissRate"]:
# for metric_label in ["L1d_TotAvgMissLat", "L2_TotAvgMissLat"]:
for metric_label in ["L1d_TotAvgMissLat"]:
    print(f"Plotting {metric_label} vs vlen")
    if metric_label in df_labels:
        # for dim in dimension_list:
        # for dim in [3072]:
        for dim in [256, 512, 1024, 2048, 3072, 4096]:
            # for top_k in top_k_list:
            for top_k in [100]:
                # plt.figure(figsize=(8, 6))
                plt.figure(figsize=(4, 3))
                max_y_value = -float('inf')
                min_y_value = float('inf')
                plot_has_data = False

                for i, config in enumerate(plot_configs):
                    df_current = config['df']
                    workload_type = config['workload']
                    label = config['label']

                    mask = (df_current['dimension'] == dim) & (df_current['top_k'] == top_k) & (df_current['workload_type'] == workload_type)
                    if mask.any():
                        data = df_current[mask].sort_values('vlen')
                        if not data.empty and metric_label in data.columns:
                            marker = markers[i % len(markers)]
                            linestyle = linestyles[i % len(linestyles)]

                            # Prepare y-values and adjust unit if necessary
                            y_values = data[metric_label]
                            y_label_text = metric_label
                            if metric_label == 'SimSecond':
                                # y_values = y_values * 1000 # Convert seconds to milliseconds
                                y_label_text = f'仿真时间 (秒)'
                            elif metric_label == 'L1d_TotAvgMissLat':
                                y_label_text = f"一级数据缓存未命中延迟 (周期)"

                            # Use plt.plot for linear x-axis
                            plt.plot(data['vlen'], y_values, 
                                     marker=marker, linestyle=linestyle, label=label, 
                                     linewidth=1, markersize=3)

                            current_max_y = y_values.max()
                            current_min_y = y_values.min()

                            if pd.notna(current_max_y) and current_max_y > max_y_value:
                                max_y_value = current_max_y
                            if pd.notna(current_min_y) and current_min_y < min_y_value:
                                min_y_value = current_min_y

                            plot_has_data = True # Mark that we added data to this plot
                        else:
                            print(f"Warning: No data or metric '{metric_label}' for {label} at dim={dim}, top_k={top_k}")
                    else:
                        print(f"Warning: No data for {label} at dim={dim}, top_k={top_k}")

                # Only proceed if the plot actually contains data
                if plot_has_data:
                    # Customize the plot
                    plt.xticks(vlen_list, fontproperties=en_font) # Set specific ticks for vlen
                    plt.yticks(fontproperties=en_font)
                    plt.xlabel("向量寄存器长度", fontproperties=zh_font)
                    plt.ylabel(y_label_text, fontproperties=zh_font) # Use updated label
                    plt.xscale('log', base=2) # Set x-axis to log scale
                    # plt.yscale('log', base=10)
                    # plt.title(f'{metric_label} vs VLEN (Dim={dim}, TopK={top_k})', fontproperties=en_font)
                    ax = plt.gca()

                    # Adjust y-axis limits based on data range, remove forcing from 0
                    if max_y_value > -float('inf') and min_y_value < float('inf'):
                        y_padding = (max_y_value - min_y_value) * 0.3 # Add 5% padding
                        # y_padding = (max_y_value - min_y_value) * 0.1 # Add 5% padding
                        if y_padding == 0: # Handle case where min == max
                            y_padding = max(abs(max_y_value * 0.1), 0.1) # Add 10% or 0.1 padding
                        plt.ylim(min_y_value - y_padding, max_y_value + y_padding)
                        # Use MaxNLocator for potentially better tick placement
                        # Check if data looks integer-like or has small range for integer ticks
                        is_integer_like = all(y_values.apply(lambda x: pd.isna(x) or x == int(x)))
                        if is_integer_like or (max_y_value - min_y_value < 10 and min_y_value > -10):
                            ax.yaxis.set_major_locator(MaxNLocator(integer=True))
                    else:
                        plt.ylim(0, 1) # Default fallback if range calculation failed

                    # Place legend outside the plot on the right side
                    # plt.legend(frameon=True, prop=en_font, loc='center left', bbox_to_anchor=(1, 0.5))
                    # plt.legend(frameon=False, prop=en_font, loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=3)
                    plt.legend(frameon=False, prop=en_font, loc='upper center', bbox_to_anchor=(0.5, 1.2), ncol=3)
                    plt.grid(axis="y", linestyle="--", linewidth=0.5, color="gray", alpha=0.6)
                    plt.tight_layout()

                    # Save the figure
                    fig_name = os.path.join(output_dir, f"gem5-{metric_label}_vs_vlen_dim{dim}_topk{top_k}.pdf")
                    csv_name = os.path.join(output_dir, f"gem5-{metric_label}_vs_vlen_dim{dim}_topk{top_k}.csv")

                    # Save the plot
                    plt.savefig(fig_name, bbox_inches='tight') # Use bbox_inches='tight' to include legend

                    # Save the data to a CSV file for all lines in the plot
                    all_data_to_save = pd.DataFrame()
                    for config in plot_configs:
                        df_current = config['df']
                        label = config['label']
                        mask = (df_current['dimension'] == dim) & (df_current['top_k'] == top_k) & (df_current['workload_type'] == config['workload'])
                        if mask.any():
                            data = df_current[mask].sort_values('vlen')
                            if not data.empty and metric_label in data.columns:
                                temp_data = data[['vlen', metric_label]].copy()
                                temp_data['label'] = label
                                all_data_to_save = pd.concat([all_data_to_save, temp_data], ignore_index=True)

                    if not all_data_to_save.empty:
                        all_data_to_save.to_csv(csv_name, index=False)
                    plt.show()
                    print(f"Plot saved as {fig_name}")
                else:
                    print(f"Skipping plot for dim={dim}, top_k={top_k} as no data was found.")
                plt.close() # Close the plot
    else:
        print(f"Metric '{metric_label}' not found in DataFrame columns.")


In [ ]:
# ... (The original loop plotting vs dimension can be kept or removed)
# Original plotting loop (vs dimension)
print("\nPlotting vs Dimension (original)")
# for metric_label in ['SimSecond', 'CPU_IPC', 'L1d_TotMissRate', 'L1d_TotAvgMissLat', 'L2_TotMissRate', 'L2_TotAvgMissLat']:
# for metric_label in ['SimSecond']:
# for metric_label in ['CPU_IPC']:
for metric_label in ['L1d_TotMissRate', "L2_TotMissRate"]:
    print(metric_label)
    if metric_label in df_labels:
        for vlen in vlen_list:
            plt.figure(figsize=(4, 3))
            max_y_value = -float('inf')
            min_y_value = float('inf')
            plot_has_data = False

            for i, config in enumerate(plot_configs):
                df_current = config['df']
                workload_type = config['workload']
                label = config['label']
                
                mask = (df_current['vlen'] == vlen) & (df_current['top_k'] == 100) & (df_current['workload_type'] == workload_type)
                if mask.any():
                    data = df_current[mask].sort_values('dimension')
                    # only plot dimension > 32
                    data = data[data['dimension'] > 128]
                    if not data.empty and metric_label in data.columns:
                        marker = markers[i % len(markers)]
                        linestyle = linestyles[i % len(linestyles)]
                        
                        # Prepare y-values and adjust unit if necessary
                        y_values = data[metric_label]
                        y_label_text = metric_label
                        if metric_label == 'SimSecond':
                            y_values = y_values * 1000 # Convert seconds to milliseconds
                            y_label_text = f'仿真时间 (ms)'
                        if metric_label == 'CPU_IPC':
                            y_label_text = f'IPC'
                        
                        plt.semilogx(data['dimension'], y_values, 
                                     marker=marker, linestyle=linestyle, label=label, 
                                     linewidth=1, markersize=3, base=2)
                        
                        current_max_y = y_values.max()
                        current_min_y = y_values.min()
                        
                        if pd.notna(current_max_y) and current_max_y > max_y_value:
                            max_y_value = current_max_y
                        if pd.notna(current_min_y) and current_min_y < min_y_value:
                             min_y_value = current_min_y
                             
                        plot_has_data = True
                    else:
                        print(f"Warning: No data or metric '{metric_label}' for {label} at vlen={vlen}")
                else:
                    print(f"Warning: No data for {label} at vlen={vlen}")

            if plot_has_data:
                # Customize the plot
                plt.xticks(fontproperties=en_font)
                plt.yticks(fontproperties=en_font)
                plt.xlabel('数据规模/向量维度', fontproperties=zh_font)
                # plt.ylabel(y_label_text, fontproperties=zh_font) # Use updated label
                plt.ylabel(y_label_text, fontproperties=en_font) # Use updated label
                # plt.xscale('log', base=2) # Set x-axis to log scale
                # plt.yscale('log', base=2)
                # plt.title(f'{metric_label} vs Dimension (vlen={vlen})', fontproperties=en_font)
                ax = plt.gca()
                
                # Adjust y-axis limits based on data range, remove forcing from 0
                if max_y_value > -float('inf') and min_y_value < float('inf'):
                    y_padding = (max_y_value - min_y_value) * 0.3 # Add 5% padding
                    if y_padding == 0: # Handle case where min == max
                        y_padding = max(abs(max_y_value * 0.1), 0.1) # Add 10% or 0.1 padding
                    plt.ylim(min_y_value - y_padding, max_y_value + y_padding)
                    # Use MaxNLocator for potentially better tick placement, especially for integers
                    is_integer_like = all(y_values.apply(lambda x: pd.isna(x) or x == int(x)))
                    if is_integer_like or (max_y_value - min_y_value < 10 and min_y_value > -10): # Heuristic for integer-like data
                         ax.yaxis.set_major_locator(MaxNLocator(integer=True))
                else:
                     plt.ylim(0, 1) # Default fallback if range calculation failed

                # Place legend outside the plot on the right side
                # plt.legend(frameon=True, prop=en_font, loc='center left', bbox_to_anchor=(1, 0.5))
                plt.legend(frameon=True, prop=en_font, loc='best', bbox_to_anchor=(1, 0.5))
                plt.grid(axis="y", linestyle="--", linewidth=0.5, color="gray", alpha=0.6)
                plt.tight_layout()
                
                # Save the figure
                fig_name = os.path.join(output_dir, f"gem5-{metric_label}_vs_dim_vlen{vlen}.pdf")
                csv_name = os.path.join(output_dir, f"gem5-{metric_label}_vs_dim_vlen{vlen}.csv")
                
                # Save the plot
                plt.savefig(fig_name, bbox_inches='tight') # Use bbox_inches='tight' to include legend
                
                # Save the data to a CSV file for all lines in the plot
                all_data_to_save = pd.DataFrame()
                for config in plot_configs:
                    df_current = config['df']
                    label = config['label']
                    mask = (df_current['vlen'] == vlen) & (df_current['top_k'] == 100) & (df_current['workload_type'] == config['workload'])
                    if mask.any():
                        data = df_current[mask].sort_values('dimension')
                        data = data[data['dimension'] > 128]  # Filter dimension > 128
                        if not data.empty and metric_label in data.columns:
                            temp_data = data[['dimension', metric_label]].copy()
                            temp_data['label'] = label
                            all_data_to_save = pd.concat([all_data_to_save, temp_data], ignore_index=True)
                
                if not all_data_to_save.empty:
                    all_data_to_save.to_csv(csv_name, index=False)
                
                plt.show()
                print(f"Plot saved as {fig_name}")
                print(f"Data saved as {csv_name}")
            else:
                 print(f"Skipping plot for vlen={vlen} as no data was found.")
            plt.close() # Close the plot to avoid displaying it inline
    else:
        print(f"Metric '{metric_label}' not found in DataFrame columns.")
